## Einleitung

In diesem Toy Projekt arbeiten wir mit Tweets von amerikanischen Politikern. Wir wollen einmal sehen, ob es möglich ist, den typischen Tweet-Style des amerikanischen Chefpolitikers mit Machine Learning zu erkennen und seine Tweets von anderen zu unterscheiden.

Dafür verwenden wir den Cloud-Dienst [Monkeylearn](https://monkeylearn.com). Monkeylearn hat sich auf Natural Language Processing spezialisiert, und bietet eine Vielzahl an vorkonfigurierten Klassifiern, die man mit eigenen Daten trainieren und verwenden kann.

Unsere Daten holen wir aus dem [Trump Twitter Archive](http://www.trumptwitterarchive.com/). Direkte Downloads von Archiven sind [hier](https://github.com/bpb27/trump-tweet-archive) und weitere Archive sind [hier](https://github.com/bpb27/political_twitter_archive).

In [ ]:
# Einige Archive sind bereits im Docker Image drin:
!ls /data/data/tweets/*

## Data Cleaning

Zuerst laden wir die Daten von Donald Trump:

In [ ]:
import json
with open('/data/data/tweets/donald_trump/condensed_2016.json', 'r') as f:
    trump_tweets = json.load(f)

Nun laden wir die Daten von Ben Carson, einem weiteren Republikaner. Wir wählen bewusst zwei Republikaner. Mit Trump und einem Demokraten wäre die Aufgabe wohl etwas zu einfach.

In [ ]:
with open('/data/data/tweets/others/realbencarson_short.json', 'r') as f:
    other_tweets = json.load(f)

Schauen wir uns die Struktur mal an:

In [ ]:
trump_tweets[0:2]

Es interessieren uns lediglich `text` und `is_retweet`. Wir säubern nun die Daten ein wenig. Zum Beispiel entfernen wir Links, @mentions, Hashtags. Dies wiederum, um dem Algorithmus die Sache nicht zu einfach zu machen.

In [ ]:
import re
def filter_and_clean_tweets(j):
    """Filter out retweets and clean remaining tweets
    downloaded from http://www.trumptwitterarchive.com/ a bit
    """
    
    # filter out retweets and extract tweet text
    tweets = [entry['text'] for entry in j if not entry['is_retweet']]

    # replace \n by space
    tweets = [re.sub(r'\n', ' ', tweet) for tweet in tweets]

    # remove double quotes
    tweets = [re.sub(r'"', '', tweet) for tweet in tweets]

    # remove leading dot
    tweets = [re.sub(r'^\.', '', tweet) for tweet in tweets]

    # remove 'RT ' at beginning
    tweets = [re.sub(r'^RT\s*', '', tweet) for tweet in tweets] 

    # remove @mentions
    tweets = [re.sub(r'@\w+:?', r'', tweet) for tweet in tweets] 

    # remove hashtags
    tweets = [re.sub(r'#\w+:?', r'', tweet) for tweet in tweets]

    # remove links (do it several times to catch them all)
    for i in range(3):
        tweets = [re.sub(r'(.*)\s*https?://.+\s*(.*)', r'\1 \2', tweet) for tweet in tweets]

    # remove whitespace from beginning and end
    tweets = [tweet.rstrip().lstrip() for tweet in tweets]

    # replace &amp; with &
    tweets = [re.sub(r'&amp;', r'&', tweet) for tweet in tweets]

    # condense multiple spaces
    tweets =[re.sub(r'\s+', r' ', tweet) for tweet in tweets]

    # return result for all tweets that are not empty now after the cleaning
    return [tweet for tweet in tweets if tweet != '']

In [ ]:
trump_tweets = filter_and_clean_tweets(trump_tweets)
other_tweets = filter_and_clean_tweets(other_tweets)

Schauen wir uns das schnell an (mit irgendwelchen zufälligen Indices)

In [ ]:
trump_tweets[42:45]

OK, nun haben wir unsere Trainingsdaten. Machen wir daraus einen Pandas DataFrame, damit können wir die Daten flexibel umformen.

Erstelle einen DataFrame mit einem Tweet pro Zeile und mit zwei Spalten, eine Spalte mit dem tweet Text aller Trump und NotTrump tweets und eine mit dem Label: für Trump-Tweets 1 und für die anderen 0.

#### Aufgabe 1

In [ ]:
# Erstelle den Pandas DataFrame wie oben beschrieben
import pandas as pd

df = pd.DataFrame(...)

df.columns=['tweet', 'label'] # verwende diese zwei Spaltennamen
df.sample(frac=0.001) # anstelle df.head(), denn damit sähen wir nur Trump tweets

#### Vorschlag zur Umsetzung

In [ ]:
import pandas as pd

# Es gibt viele Möglichkeiten, wie man diesen DataFrame erstellen kann.
# Hier eine kompakte, auch wenn vielleicht nicht die leserlichste
# Das .T am schluss dreht den DataFrame (.T für transpose)
df = pd.DataFrame([trump_tweets+other_tweets, [1]*len(trump_tweets)+[0]*len(other_tweets)]).T

df.columns=['tweet', 'label']
df.sample(frac=0.001) # anstelle df.head(), denn damit sähen wir nur Trump tweets

Nun schauen wir uns das API von Monkeylearn an. Einige Beispiele des Python-APIs sind [hier](https://github.com/monkeylearn/monkeylearn-python), und hier ist die gesamte [API Referenz](https://monkeylearn.com/docs/article/api-reference/).

Wir initialisieren das API und ertstellen ein Modell. Wähle im untenstehenden Code einen eigenen Classifier-Namen, möglichst so, dass die anderen Workshop-Teilnehmer nicht den gleichen Namen erwischen. Führe dann den Code aus.

In [ ]:
from monkeylearn import MonkeyLearn

API_KEY = 'XXX'

# Erstelle ein ml Objekt
ml = MonkeyLearn(API_KEY)

# Erstelle einen Klassifier. ACHTUNG: Mit dem zur Verfügung stehenden API Key
# können maximal 3 Classifier gleichzeitig erstellt werden!
res = ml.classifiers.create(übergib hier einen eigenen String)

Mit dem Account, den wir verwenden, können genau 12 Classifiers erstellt werden. Bitte erstelle deshalb nur einen Classifier, damit die anderen Workshop-Teilnehmer auch einen machen können. Musst Du einen Classifier löschen, melde Dich bei mir.

Schau Dir nun das oben verlinkten Beispiel (github) an und vervollständige den folgenden Code:

#### Aufgabe 2

In [ ]:
# Vervollständige

# Hohl die ID des neuen Moduls
module_id = ...

# Hohl den Root Node
root_id = ...

# Erstelle zwei neue Kategorien positive_id und negative_id mit den Bezeichnern 'Trump' (pos) und NotTrump' (neg)
...
positive_id = ...
negative_id = ...

#### Vorschlag zur Umsetzung

In [ ]:
# Hohl die ID des neuen Moduls
module_id = res.result['classifier']['hashed_id']

# Hohl den Root Node
res = ml.classifiers.detail(module_id)
root_id = res.result['sandbox_categories'][0]['id']

# Erstelle zwei neue Kategorien mit den Bezeichnern 'Trump' und NotTrump'
res = ml.classifiers.categories.create(module_id, 'Trump', root_id)
positive_id = res.result['category']['id']
res = ml.classifiers.categories.create(module_id, 'NotTrump', root_id)
negative_id = res.result['category']['id']

Wir müssen in unserem DataFrame die Labels (bisher 1 für Trump und 0 für Nicht-Trump) durch die obigen `positive_id`  und `negative_id` ersetzen.

#### Aufgabe 3

In [ ]:
# ändere im DataFrame 'trump' in positive_id und 'other' in negative_id
...

#### Vorschlag zur Umsetzung

In [ ]:
# Auch hier wieder diverse Möglichkeiten, hier eine davon
df['label'] = df['label'].map(lambda x: positive_id if x==1 else negative_id)

Nun teilen wir wie im follow-along Beispiel unsere Daten in ein Trainings- und ein Validierungsset auf.

#### Aufgabe 4

In [ ]:
# Teile die Daten in 80% Trainingsdaten und 20% Validierungsdaten
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = ...

#### Vorschlag zur Umsetzung

In [ ]:
from sklearn.model_selection import train_test_split
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    X_train, X_test, y_train, y_test = train_test_split(df['tweet'], df['label'], train_size=0.8, stratify=df['label']) 

Da wir deutlich mehr Tweets von Trump haben als andere, müssen wir dafür sorgen, dass dieses Verhältnis innerhalb der Aufteilungen in Trainingsdaten und Validierungsdaten gleich bleibt. Dies erledigt der Parameter `stratified`.

Das Monkeylearn API verlangt die Samples für das Training als Liste, deren Elemente 2-Tupel sind.
Ein solches Tupel beinhaltet den Text und eine der beiden oben generierten IDs, also zum Beispiel `('trump tweet text', positive_id)`.

Nun haben wir aber dummerweise während dem Umformen den Datentyp des Labels geändert:

In [ ]:
(type(positive_id), type(y_train[0]))

Das müssen wir wieder ändern, da Monkeylearn nicht mit Numpy Datentypen umgehen kann. Ich werde das als Verbesserungsvorschlag melden.

In [ ]:
y_train_int = [i.item() for i in y_train.values]
del y_train
y_train = y_train_int
y_test_int = [i.item() for i in y_test.values]
del y_test
y_test = y_test_int

#### Aufgabe 5

In [ ]:
# Erstelle eine Liste aus X_train und y_train

samples = ...

#### Vorschlag zur Umsetzung

In [ ]:
# Auch hier gibt es viele Wege, zum Ziel zu kommen. Der kürzeste:
samples = zip(X_train, y_train_int)

Nun können wir den Classifier trainieren.

In [ ]:
# Samples uploaden
# auch hier etwas unschön, dass Monkeylearn keine Iteratoren und nur Listen annimmt
res = ml.classifiers.upload_samples(module_id, list(samples))

# Trainieren
res = ml.classifiers.train(module_id)

Nun validieren wir das Modell mit unserem Validierungsset. Orientiere Dich wiederm am Beispiel wie vorher.

**Achtung**: Da die verfügbaren Requests beschränkt sind, pass bitte auf, dass Du keine Endlosloops baust. Für den gesamten Workshop stehen für alle Teilnehmer zusammen 150'000 Queries zur Verfügung, was eigentlich vorig reichen sollte.

#### Aufgabe 6

In [ ]:
# Mache predictions für das ganze Validierungsset. Speichere sie vorerst so, wie sie von Monkeylearn zurückkommen.
predictions = ...

#### Vorschlag zur Umsetzung

In [ ]:
predictions = ml.classifiers.classify(module_id, X_test, sandbox=True).result

Ok, eine Prediction sieht so aus:

In [ ]:
predictions[0]

Das ist etwas unpraktisch, extrahieren wir das.

#### Aufgabe 7

In [ ]:
# Extrahiere die vorhergesagte category_id aus den predictions in eine separate Liste
pred = ...

#### Vorschlag zur Umsetzung

In [ ]:
pred = [p[0]['category_id'] for p in predictions]

Nun berechnen wir die Accuracy.

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)

Nicht schlecht! 88% der Samples wurden korrekt klassifiziert. Damit wären wir am Ende dieses Toy Projects.

Wenn Du möchtest, kannst nun etwas weiter experimentieren.

Aufschlussreich ist es beispielsweise, sich einmal das Dashboard von [Monkeylearn](https://monkeylearn.com) anzuschauen. Dazu müsstest Du Dich aber selber dort registrieren. Für den Free Tier braucht es lediglich eine Email-Adresse, damit kannst Du Modelle mit maximal 3000 Samples trainieren und 1000 Requests machen. Trainieren zählt nicht zu den Requests.

Du kannst auch auf den Team Tier upgraden, dann bekommst Du 300'000 Requests. Dafür benötigst Du aber eine Kreditkarte, und nach 14 Tagen wird diese belastet, wenn Du vorher nicht kündigst.

Alternativ kannst Du das Dashboard und Deinen Klassifier auch kurz auf meinem Laptop anschauen.

----

Weitere Ideen:
* Ein paar neuere Trump tweets klassifizieren und schauen, ob unser Detektor diese auch als von Trump stammend erkennt. Tweets von 2017 sind im Image vorhanden.
* Tweets von jemand anderes als Trump und Ben Carson klassifizieren und schauen, ob unser Detektor diese auch als Nicht-Trump-Tweets erkennt
* Hillary Clinton hinyunehmen und zwischen allen dreien unterscheiden. Tip: Das Modell muss nicht neu erstellt werden, es reicht, nur die neuen Tweets von Hillary hinzuzufügen und nochmals zu trainieren.

**Wichtig**: Wenn Du Deinen Classifier nicht mehr brauchst, so lösche ihn doch bitte gleich:

In [ ]:
ml.classifiers.delete(module_id)